In [2]:
from dotenv import load_dotenv

In [ ]:
env_loaded = load_dotenv('../../.env-local')
env_loaded

In [ ]:
from datadocai.models import CurrentTable

TRINO_CATALOG = 'postgres'
TRINO_SCHEMA = 'public'
TRINO_TABLE = 'house_pricing'

# construct the table you want to analyse
ct = CurrentTable(trino_catalog=TRINO_CATALOG,
                  trino_schema=TRINO_SCHEMA,
                  trino_table=TRINO_TABLE)

In [ ]:
import os
from datadocai.database import DatabaseClient

# connect to trino
dc = DatabaseClient(host=os.getenv('TRINO_HOST'),
                    port=os.getenv('TRINO_PORT'),
                    user=os.getenv('TRINO_USER'),
                    password=os.getenv('TRINO_PASSWORD'))

In [6]:
from langchain_openai.chat_models import AzureChatOpenAI
from httpx import Client

llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    openai_api_version="2024-05-01-preview",
    streaming=True,
    verbose=True,
    http_client=Client(verify=False)
)

In [ ]:
from datadocai.metadata import TableMetadataManager
from datadocai.metadata.exporter.json import MetadataJsonExporter

metadata_exporter = MetadataJsonExporter(current_table=ct)

tmm = TableMetadataManager(current_table=ct, database_client=dc, llm=llm, metadata_exporter=metadata_exporter)

In [ ]:
# launch the process
result, exporter = tmm.process()